### This notebook explores Pivot Algorithm with GPU (both Atmosphere Statistics method and Recursive Method)

In [1]:
!pip install cupy-cuda11x

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 MB 17.3 MB/s eta 0:00:00:00:010:01


### 1. Atmosphere Statistics Method

### Key implementation details:
1. common base thermolisation on CPU, then parallel pivot moves will be done to the base thermolised walk.
2. max_trial: max trials pivot_move() will commit before it fails to generate a new SAW and returns the same walk. This prevents the appearance of many repetitive walks.
3. pivot_steps_mode has 2 different modes, under 'multiple' mode, pivot moves will be executed for L times before samling the atmosphere value, to ensure the independence of $n$ sample from the base thermolised walk.
4. uses cp.bincount() to achieve the collision check at computation time complexity = $O(N)$.
5. the SAWs will be initialized at starting point (L_max, L_max) and the 2d coordinates will be mapped to positive integers to fullfil the prerequisites of cp.bincount() method.

In [1]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from scipy.optimize import curve_fit

# CUDA kernel for parallel pivot move (no offset needed)
pivot_kernel = cp.RawKernel(r'''
extern "C" __global__
void pivot_move_kernel(
    int* walks,  // Flattened array: [num_samples, L+1, 2]
    int num_samples, int L, int grid_size, int max_trial,
    int* counts,  // Global array: [num_samples, grid_size * grid_size]
    int* k_values,  // Random pivot points
    int* g_values,  // Random symmetry operations
    int* collision_flags  // Output: 1 if successful pivot, 0 if failed
) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid >= num_samples) return;

    int walk_offset = tid * (L + 1) * 2;  // the starting position of current thread's walk
    int count_offset = tid * grid_size * grid_size;  // the starting position of current thread's count

    // Get random pivot point and symmetry operation
    int k = k_values[tid];
    int g = g_values[tid];

    // Try pivot move up to max_trial times
    for (int trial = 0; trial < max_trial; trial++) {
        // Copy walk to temporary array
        int temp_walk[2002][2];
        for (int i = 0; i <= L; i++) {
            temp_walk[i][0] = walks[walk_offset + i * 2];
            temp_walk[i][1] = walks[walk_offset + i * 2 + 1];
        }

        // Apply symmetry operation to subwalk (from k to L)
        int pivot_x = temp_walk[k][0];
        int pivot_y = temp_walk[k][1];
        for (int i = k; i <= L; i++) {
            int dx = temp_walk[i][0] - pivot_x;
            int dy = temp_walk[i][1] - pivot_y;
            if (g == 0) {  // rotate90
                temp_walk[i][0] = pivot_x - dy;
                temp_walk[i][1] = pivot_y + dx;
            } else if (g == 1) {  // rotate180
                temp_walk[i][0] = pivot_x - dx;
                temp_walk[i][1] = pivot_y - dy;
            } else if (g == 2) {  // reflect_x
                temp_walk[i][0] = pivot_x + dx;
                temp_walk[i][1] = pivot_y - dy;
            } else {  // reflect_y
                temp_walk[i][0] = pivot_x - dx;
                temp_walk[i][1] = pivot_y + dy;
            }
        }

        // Collision check 
        bool has_collision = false;
        for (int i = 0; i <= L; i++) {
            int x = temp_walk[i][0];
            int y = temp_walk[i][1];
            if (x < 0 || x >= grid_size || y < 0 || y >= grid_size) {
                has_collision = true;
                break;
            }
            int idx = count_offset + (x * grid_size + y);
            atomicAdd(&counts[idx], 1);
            if (counts[idx] > 1) {
                has_collision = true;
                break;
            }
        }

        // Reset counts and update walk if no collision
        if (!has_collision) {
            for (int i = 0; i <= L; i++) {
                int x = temp_walk[i][0];
                int y = temp_walk[i][1];
                int idx = count_offset + (x * grid_size + y);
                counts[idx] = 0;
            }
            for (int i = 0; i <= L; i++) {
                walks[walk_offset + i * 2] = temp_walk[i][0];
                walks[walk_offset + i * 2 + 1] = temp_walk[i][1];
            }
            collision_flags[tid] = 1;
            return;
        } else {
            for (int i = 0; i <= L; i++) {
                int x = temp_walk[i][0];
                int y = temp_walk[i][1];
                int idx = count_offset + (x * grid_size + y);
                counts[idx] = 0;
            }
            // Update k and g for next trial
            k = (k + 1) % (L - 1);
            if (k == 0) k = 1;
            g = (g + 1) % 4;
        }
    }
    collision_flags[tid] = 0;
}
''', 'pivot_move_kernel')

# CUDA kernel for parallel atmosphere calculation (no offset needed)
atmosphere_kernel = cp.RawKernel(r'''
extern "C" __global__
void atmosphere_kernel(
    int* walks,  // Flattened array: [num_samples, L+1, 2]
    int num_samples, int L, int grid_size,
    int* counts,  // Global array: [num_samples, grid_size * grid_size]
    float* atm_counts  // Output: atmosphere counts for each walk
) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid >= num_samples) return;

    int walk_offset = tid * (L + 1) * 2;  // the starting position of current thread's walk
    int count_offset = tid * grid_size * grid_size;  // the starting position of current thread's count

    // Build counts for the walk
    for (int i = 0; i <= L; i++) {
        int x = walks[walk_offset + i * 2];
        int y = walks[walk_offset + i * 2 + 1];
        int idx = count_offset + (x * grid_size + y);
        atomicAdd(&counts[idx], 1);
    }

    // Last point of the walk
    int last_x = walks[walk_offset + L * 2];
    int last_y = walks[walk_offset + L * 2 + 1];

    // Check 4 directions
    int atm_count = 0;
    int directions[4][2] = {{1, 0}, {-1, 0}, {0, 1}, {0, -1}};
    for (int d = 0; d < 4; d++) {
        int new_x = last_x + directions[d][0];
        int new_y = last_y + directions[d][1];
        if (new_x < 0 || new_x >= grid_size || new_y < 0 || new_y >= grid_size) continue;
        int idx = count_offset + (new_x * grid_size + new_y);
        if (counts[idx] == 0) atm_count++;
    }

    // Reset counts
    for (int i = 0; i <= L; i++) {
        int x = walks[walk_offset + i * 2];
        int y = walks[walk_offset + i * 2 + 1];
        int idx = count_offset + (x * grid_size + y);
        counts[idx] = 0;
    }

    atm_counts[tid] = (float)atm_count;
}
''', 'atmosphere_kernel')

class SAWSimulatorGPU:
    def __init__(self, L_max=1000):
        """Initialize SAW simulator for 2D square lattice on GPU"""
        self.directions = np.array([[1,0], [-1,0], [0,1], [0,-1]])  # Right, Left, Up, Down
        self.L_max = L_max
        self.grid_size = 2 * L_max + 1  # Grid size to accommodate coordinates [0, 2*L_max]
        print(f"Grid size: {self.grid_size}")

    def generate_initial_saw(self, L):
        """Create a single straight initial SAW starting at (L_max, L_max)"""
        start_x, start_y = self.L_max, self.L_max
        walk = np.array([[start_x + i, start_y] for i in range(L+1)], dtype=np.int32)
        return walk

    def generate_multiple_saws(self, base_walk, num_samples):
        """Create num_samples copies of the base walk"""
        walks = np.tile(base_walk[np.newaxis, :, :], (num_samples, 1, 1))
        return cp.array(walks)  # Shape: (num_samples, L+1, 2)

    def pivot_move_single(self, walk, L, max_trial):
        """Perform a single pivot move on a single walk on CPU using np.bincount"""
        for _ in range(max_trial):
            k = np.random.randint(1, L)  # Random pivot point
            g = np.random.randint(0, 4)  # Random symmetry operation (0: rotate90, 1: rotate180, 2: reflect_x, 3: reflect_y)

            # Apply symmetry operation to subwalk
            subwalk = walk[k:] - walk[k]
            if g == 0:  # rotate90
                transformed = np.array([[-y, x] for [x, y] in subwalk])
            elif g == 1:  # rotate180
                transformed = np.array([[-x, -y] for [x, y] in subwalk])
            elif g == 2:  # reflect_x
                transformed = np.array([[x, -y] for [x, y] in subwalk])
            else:  # reflect_y
                transformed = np.array([[-x, y] for [x, y] in subwalk])

            new_walk = np.concatenate([walk[:k], walk[k] + transformed])

            # Collision check using np.bincount on CPU
            indices = new_walk[:, 0] * self.grid_size + new_walk[:, 1]
            counts = np.bincount(indices, minlength=self.grid_size * self.grid_size)
            if np.all(counts <= 1):
                return new_walk, True

        return walk, False

    def pivot_move_parallel(self, walks, L, max_trial):
        """Perform pivot move on all walks in parallel using CUDA kernel"""
        num_samples = walks.shape[0]
        
        walks = walks.astype(cp.int32)
        walks_flat = walks.reshape(num_samples * (L + 1) * 2)
        collision_flags = cp.zeros(num_samples, dtype=cp.int32)
        
        counts = cp.zeros(num_samples * self.grid_size * self.grid_size, dtype=cp.int32)

        k_values = cp.random.randint(1, L, size=num_samples, dtype=cp.int32)
        g_values = cp.random.randint(0, 4, size=num_samples, dtype=cp.int32)

        threads_per_block = min(256, num_samples)
        blocks_per_grid = (num_samples + threads_per_block - 1) // threads_per_block
        
        pivot_kernel(
            (blocks_per_grid,), (threads_per_block,),
            (walks_flat, num_samples, L, self.grid_size, max_trial, counts, k_values, g_values, collision_flags)
        )

        walks = walks_flat.reshape(num_samples, L + 1, 2)
        return walks, collision_flags

    def compute_atmosphere_parallel(self, walks, L, num_samples):
        """Compute atmosphere counts for all walks in parallel using CUDA kernel"""
        walks_flat = walks.reshape(num_samples * (L + 1) * 2)
        atm_counts = cp.zeros(num_samples, dtype=cp.float32)
        counts = cp.zeros(num_samples * self.grid_size * self.grid_size, dtype=cp.int32)

        threads_per_block = min(256, num_samples)
        blocks_per_grid = (num_samples + threads_per_block - 1) // threads_per_block

        atmosphere_kernel(
            (blocks_per_grid,), (threads_per_block,),
            (walks_flat, num_samples, L, self.grid_size, counts, atm_counts)
        )

        return atm_counts

    def simulate_atmosphere(self, L, num_samples, max_trial, pivot_steps=1):
        """Estimate atmosphere statistics for walks of length L using GPU parallelization
        pivot_steps: number of pivot moves per sample (1 or L)
        """
        # Validate pivot_steps
        if pivot_steps not in [1, L]:
            raise ValueError(f"pivot_steps must be 1 or L ({L}), got {pivot_steps}")

        # Generate a single initial walk
        walk = self.generate_initial_saw(L)
        
        # Thermalization (burn-in) on a single walk on CPU
        for _ in range(10 * L):
            walk, _ = self.pivot_move_single(walk, L, max_trial)
        
        # Generate num_samples copies of the thermalized walk
        walks = self.generate_multiple_saws(walk, num_samples)

        # Production run: perform pivot_steps pivot moves per walk
        for _ in range(pivot_steps):
            walks, _ = self.pivot_move_parallel(walks, L, max_trial)
        
        # Compute atmosphere for all walks
        atm_counts = self.compute_atmosphere_parallel(walks, L, num_samples)
        
        atm_counts = atm_counts.get()
        return np.mean(atm_counts), np.std(atm_counts) / np.sqrt(len(atm_counts))
    
    def estimate_mu(self, num_samples=10000, max_trial=10, pivot_steps_mode='single'):
        """Estimate μ using atmosphere statistics
        max_trial: max trials before pivot_move() fails to generate a new SAW and returns the same walk 
        pivot_steps_mode: 'single' (1 pivot) or 'multiple' (L pivots)
        """
        lengths = np.arange(4, self.L_max+1, 2)
        atm_means = []
        atm_stds = []
        
        print("Running SAW simulations for μ estimation on GPU...")
        start = time()
        for L in tqdm(lengths):
            pivot_steps = 1 if pivot_steps_mode == 'single' else L
            mean_atm, std_atm = self.simulate_atmosphere(L, num_samples, max_trial, pivot_steps=pivot_steps)
            atm_means.append(mean_atm)
            atm_stds.append(std_atm)
            print(f"L={L}: ⟨a⟩ = {mean_atm:.3f} ± {std_atm:.3f}")
        end = time()
        
        atm_means = np.array(atm_means)
        atm_stds = np.array(atm_stds)
        def model(n, mu, gamma, c):
            return mu * (1 + (gamma-1)/n + c/n**2)
        
        popt, pcov = curve_fit(model, lengths, atm_means)
        mu, gamma = popt[0], popt[1]
        
        # Plot 1: <a> vs L
        plt.figure(figsize=(10,6))
        plt.scatter(lengths, atm_means, label='Simulation Data')
        x_fit = np.linspace(lengths[0], lengths[-1], 100)
        plt.plot(x_fit, model(x_fit, *popt), 'r-', 
                label=f'Fit: μ={mu:.6f}, γ={gamma:.6f}')
        plt.xlabel('L')
        plt.ylabel('<a>')
        plt.title(f'Atmosphere Statistics for 2D SAWs (GPU Parallel), N={num_samples}, Time={end-start:.2f}s, mu~{mu:.4f}')
        plt.legend()
        plt.grid(True)
        plt.savefig(f'saw_mu_estimation_gpu_parallel_L_{pivot_steps_mode}.png', dpi=300)
        plt.close()
        
        # Plot 2: <a> vs 1/L with benchmark mu
        benchmark_mu = 2.638158  # Theoretical mu for 2D SAW
        inv_lengths = 1 / lengths
        plt.figure(figsize=(10,6))
        plt.scatter(inv_lengths, atm_means, label='Simulation Data')
        x_fit_inv = np.linspace(0, inv_lengths[0], 100)
        plt.plot(x_fit_inv, model(1/x_fit_inv, *popt), 'r-', 
                label=f'Fit: μ={mu:.6f}, γ={gamma:.6f}')
        plt.axhline(y=benchmark_mu, color='g', linestyle='--', label=f'Benchmark μ={benchmark_mu:.6f}')
        plt.xlabel('1/L')
        plt.ylabel('<a>')
        plt.title(f'Atmosphere Statistics for 2D SAWs: Convergence to μ (GPU Parallel), N={num_samples}, Time={end-start:.2f}s, mu~{mu:.4f}')
        plt.legend()
        plt.grid(True)
        plt.savefig(f'saw_mu_estimation_gpu_parallel_inv_L_{pivot_steps_mode}.png', dpi=300)
        plt.close()
        
        return mu, gamma

# Benchmarking
if __name__ == "__main__":
    simulator = SAWSimulatorGPU(L_max=200)
    
    # Test both modes
    for mode in ['single', 'multiple']:
        print(f"\nTesting mode: {mode}")
        start_time = time()
        mu, gamma = simulator.estimate_mu(num_samples=10000, max_trial=10, pivot_steps_mode=mode)
        gpu_time = time() - start_time
        
        print(f"GPU Parallel Version Results (Mode: {mode}):")
        print(f"Estimated μ = {mu:.6f}")
        print(f"Estimated γ = {gamma:.6f}")
        print(f"Computation Time = {gpu_time:.2f} seconds")

Grid size: 401

Testing mode: single
Running SAW simulations for μ estimation on GPU...


  3%|▎         | 3/99 [00:02<00:51,  1.85it/s]

L=4: ⟨a⟩ = 2.753 ± 0.004
L=6: ⟨a⟩ = 3.000 ± 0.000
L=8: ⟨a⟩ = 2.931 ± 0.003
L=10: ⟨a⟩ = 2.832 ± 0.004


  7%|▋         | 7/99 [00:02<00:19,  4.83it/s]

L=12: ⟨a⟩ = 2.797 ± 0.005
L=14: ⟨a⟩ = 2.980 ± 0.001
L=16: ⟨a⟩ = 2.963 ± 0.002


...

 99%|█████████▉| 98/99 [05:20<00:08,  8.14s/it]

L=198: ⟨a⟩ = 2.856 ± 0.004


100%|██████████| 99/99 [05:29<00:00,  3.33s/it]

L=200: ⟨a⟩ = 2.826 ± 0.005


GPU Parallel Version Results (Mode: multiple):
Estimated μ = 2.646908
Estimated γ = 1.646891
Computation Time = 330.74 seconds


The variable 'counts' is of $numSamples \times gridSize^2 \times 4$, thus the simulation will fail when large numSamples and gridSize make the size of 'counts' exceed GPU's memory.

Again, we use batch execution to solve this issue.

In [ ]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from scipy.optimize import curve_fit

# CUDA kernel for parallel pivot move (no offset needed)
pivot_kernel = cp.RawKernel(r'''
extern "C" __global__
void pivot_move_kernel(
    int* walks,  // Flattened array: [batch_size, L+1, 2]
    int batch_size, int L, int grid_size, int max_trial,
    int* counts,  // Global array: [batch_size, grid_size * grid_size]
    int* k_values,  // Random pivot points
    int* g_values,  // Random symmetry operations
    int* collision_flags  // Output: 1 if successful pivot, 0 if failed
) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid >= batch_size) return;

    int walk_offset = tid * (L + 1) * 2;
    int count_offset = tid * grid_size * grid_size;

    // Get random pivot point and symmetry operation
    int k = k_values[tid];
    int g = g_values[tid];

    // Try pivot move up to max_trial times
    for (int trial = 0; trial < max_trial; trial++) {
        // Copy walk to temporary array
        int temp_walk[2002][2];  // Increased size to accommodate L_max=1000
        for (int i = 0; i <= L; i++) {
            temp_walk[i][0] = walks[walk_offset + i * 2];
            temp_walk[i][1] = walks[walk_offset + i * 2 + 1];
        }

        // Apply symmetry operation to subwalk (from k to L)
        int pivot_x = temp_walk[k][0];
        int pivot_y = temp_walk[k][1];
        for (int i = k; i <= L; i++) {
            int dx = temp_walk[i][0] - pivot_x;
            int dy = temp_walk[i][1] - pivot_y;
            if (g == 0) {  // rotate90
                temp_walk[i][0] = pivot_x - dy;
                temp_walk[i][1] = pivot_y + dx;
            } else if (g == 1) {  // rotate180
                temp_walk[i][0] = pivot_x - dx;
                temp_walk[i][1] = pivot_y - dy;
            } else if (g == 2) {  // reflect_x
                temp_walk[i][0] = pivot_x + dx;
                temp_walk[i][1] = pivot_y - dy;
            } else {  // reflect_y
                temp_walk[i][0] = pivot_x - dx;
                temp_walk[i][1] = pivot_y + dy;
            }
        }

        // Collision check 
        bool has_collision = false;
        for (int i = 0; i <= L; i++) {
            int x = temp_walk[i][0];
            int y = temp_walk[i][1];
            if (x < 0 || x >= grid_size || y < 0 || y >= grid_size) {
                has_collision = true;
                break;
            }
            int idx = count_offset + (x * grid_size + y);
            atomicAdd(&counts[idx], 1);
            if (counts[idx] > 1) {
                has_collision = true;
                break;
            }
        }

        // Reset counts and update walk if no collision
        if (!has_collision) {
            for (int i = 0; i <= L; i++) {
                int x = temp_walk[i][0];
                int y = temp_walk[i][1];
                int idx = count_offset + (x * grid_size + y);
                counts[idx] = 0;
            }
            for (int i = 0; i <= L; i++) {
                walks[walk_offset + i * 2] = temp_walk[i][0];
                walks[walk_offset + i * 2 + 1] = temp_walk[i][1];
            }
            collision_flags[tid] = 1;
            return;
        } else {
            for (int i = 0; i <= L; i++) {
                int x = temp_walk[i][0];
                int y = temp_walk[i][1];
                int idx = count_offset + (x * grid_size + y);
                counts[idx] = 0;
            }
            // Update k and g for next trial
            k = (k + 1) % (L - 1);
            if (k == 0) k = 1;
            g = (g + 1) % 4;
        }
    }
    collision_flags[tid] = 0;
}
''', 'pivot_move_kernel')

# CUDA kernel for parallel atmosphere calculation (no offset needed)
atmosphere_kernel = cp.RawKernel(r'''
extern "C" __global__
void atmosphere_kernel(
    int* walks,  // Flattened array: [batch_size, L+1, 2]
    int batch_size, int L, int grid_size,
    int* counts,  // Global array: [batch_size, grid_size * grid_size]
    float* atm_counts  // Output: atmosphere counts for each walk
) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid >= batch_size) return;

    int walk_offset = tid * (L + 1) * 2;
    int count_offset = tid * grid_size * grid_size;

    // Build counts for the walk
    for (int i = 0; i <= L; i++) {
        int x = walks[walk_offset + i * 2];
        int y = walks[walk_offset + i * 2 + 1];
        int idx = count_offset + (x * grid_size + y);
        atomicAdd(&counts[idx], 1);
    }

    // Last point of the walk
    int last_x = walks[walk_offset + L * 2];
    int last_y = walks[walk_offset + L * 2 + 1];

    // Check 4 directions
    int atm_count = 0;
    int directions[4][2] = {{1, 0}, {-1, 0}, {0, 1}, {0, -1}};
    for (int d = 0; d < 4; d++) {
        int new_x = last_x + directions[d][0];
        int new_y = last_y + directions[d][1];
        if (new_x < 0 || new_x >= grid_size || new_y < 0 || new_y >= grid_size) continue;
        int idx = count_offset + (new_x * grid_size + new_y);
        if (counts[idx] == 0) atm_count++;
    }

    // Reset counts
    for (int i = 0; i <= L; i++) {
        int x = walks[walk_offset + i * 2];
        int y = walks[walk_offset + i * 2 + 1];
        int idx = count_offset + (x * grid_size + y);
        counts[idx] = 0;
    }

    atm_counts[tid] = (float)atm_count;
}
''', 'atmosphere_kernel')

class SAWSimulatorGPU:
    def __init__(self, L_max=1000):
        """Initialize SAW simulator for 2D square lattice on GPU"""
        self.directions = np.array([[1,0], [-1,0], [0,1], [0,-1]])  # Right, Left, Up, Down
        self.L_max = L_max
        self.grid_size = 2 * L_max + 1  # Grid size to accommodate coordinates [0, 2*L_max]
        self.batch_size = 256  # Set batch size to a reasonable value
        print(f"Grid size: {self.grid_size}, Batch size: {self.batch_size}")

    def generate_initial_saw(self, L):
        """Create a single straight initial SAW starting at (L_max, L_max)"""
        start_x, start_y = self.L_max, self.L_max
        walk = np.array([[start_x + i, start_y] for i in range(L+1)], dtype=np.int32)
        return walk

    def generate_multiple_saws(self, base_walk, num_samples):
        """Create num_samples copies of the base walk"""
        walks = np.tile(base_walk[np.newaxis, :, :], (num_samples, 1, 1))
        return cp.array(walks)  # Shape: (num_samples, L+1, 2)

    def pivot_move_single(self, walk, L, max_trial):
        """Perform a single pivot move on a single walk on CPU using np.bincount"""
        for _ in range(max_trial):
            k = np.random.randint(1, L)  # Random pivot point
            g = np.random.randint(0, 4)  # Random symmetry operation (0: rotate90, 1: rotate180, 2: reflect_x, 3: reflect_y)

            # Apply symmetry operation to subwalk
            subwalk = walk[k:] - walk[k]
            if g == 0:  # rotate90
                transformed = np.array([[-y, x] for [x, y] in subwalk])
            elif g == 1:  # rotate180
                transformed = np.array([[-x, -y] for [x, y] in subwalk])
            elif g == 2:  # reflect_x
                transformed = np.array([[x, -y] for [x, y] in subwalk])
            else:  # reflect_y
                transformed = np.array([[-x, y] for [x, y] in subwalk])

            new_walk = np.concatenate([walk[:k], walk[k] + transformed])

            # Collision check using np.bincount on CPU
            indices = new_walk[:, 0] * self.grid_size + new_walk[:, 1]
            counts = np.bincount(indices, minlength=self.grid_size * self.grid_size)
            if np.all(counts <= 1):
                return new_walk, True

        return walk, False

    def pivot_move_parallel(self, walks, L, max_trial):
        """Perform pivot move on all walks in parallel using CUDA kernel with batching"""
        num_samples = walks.shape[0]
        batch_size = self.batch_size
        all_collision_flags = cp.zeros(num_samples, dtype=cp.int32)

        for batch_start in range(0, num_samples, batch_size):
            batch_end = min(batch_start + batch_size, num_samples)
            current_batch_size = batch_end - batch_start

            # Extract the current batch of walks
            batch_walks = walks[batch_start:batch_end]
            batch_walks_flat = batch_walks.reshape(current_batch_size * (L + 1) * 2)
            collision_flags = cp.zeros(current_batch_size, dtype=cp.int32)

            counts = cp.zeros(current_batch_size * self.grid_size * self.grid_size, dtype=cp.int32)
            k_values = cp.random.randint(1, L, size=current_batch_size, dtype=cp.int32)
            g_values = cp.random.randint(0, 4, size=current_batch_size, dtype=cp.int32)

            threads_per_block = min(256, current_batch_size)
            blocks_per_grid = (current_batch_size + threads_per_block - 1) // threads_per_block

            pivot_kernel(
                (blocks_per_grid,), (threads_per_block,),
                (batch_walks_flat, current_batch_size, L, self.grid_size, max_trial, counts, k_values, g_values, collision_flags)
            )

            # Update walks with the batch results
            walks[batch_start:batch_end] = batch_walks_flat.reshape(current_batch_size, L + 1, 2)
            all_collision_flags[batch_start:batch_end] = collision_flags

            # Free temporary GPU memory
            del batch_walks, batch_walks_flat, collision_flags, counts, k_values, g_values
            cp.get_default_memory_pool().free_all_blocks()

        return walks, all_collision_flags

    def compute_atmosphere_parallel(self, walks, L, num_samples):
        """Compute atmosphere counts for all walks in parallel using CUDA kernel with batching"""
        batch_size = self.batch_size
        all_atm_counts = cp.zeros(num_samples, dtype=cp.float32)

        for batch_start in range(0, num_samples, batch_size):
            batch_end = min(batch_start + batch_size, num_samples)
            current_batch_size = batch_end - batch_start

            # Extract the current batch of walks
            batch_walks = walks[batch_start:batch_end]
            batch_walks_flat = batch_walks.reshape(current_batch_size * (L + 1) * 2)
            atm_counts = cp.zeros(current_batch_size, dtype=cp.float32)
            counts = cp.zeros(current_batch_size * self.grid_size * self.grid_size, dtype=cp.int32)

            threads_per_block = min(256, current_batch_size)
            blocks_per_grid = (current_batch_size + threads_per_block - 1) // threads_per_block

            atmosphere_kernel(
                (blocks_per_grid,), (threads_per_block,),
                (batch_walks_flat, current_batch_size, L, self.grid_size, counts, atm_counts)
            )

            all_atm_counts[batch_start:batch_end] = atm_counts

            # Free temporary GPU memory
            del batch_walks, batch_walks_flat, atm_counts, counts
            cp.get_default_memory_pool().free_all_blocks()

        return all_atm_counts

    def simulate_atmosphere(self, L, num_samples, max_trial, pivot_steps=1):
        """Estimate atmosphere statistics for walks of length L using GPU parallelization
        pivot_steps: number of pivot moves per sample (1 or L)
        """
        # Validate pivot_steps
        if pivot_steps not in [1, L]:
            raise ValueError(f"pivot_steps must be 1 or L ({L}), got {pivot_steps}")

        # Generate a single initial walk
        walk = self.generate_initial_saw(L)
        
        # Thermalization (burn-in) on a single walk on CPU
        for _ in range(10 * L):
            walk, _ = self.pivot_move_single(walk, L, max_trial)
        
        # Generate num_samples copies of the thermalized walk
        walks = self.generate_multiple_saws(walk, num_samples)

        # Production run: perform pivot_steps pivot moves per walk
        for _ in range(pivot_steps):
            walks, _ = self.pivot_move_parallel(walks, L, max_trial)
        
        # Compute atmosphere for all walks
        atm_counts = self.compute_atmosphere_parallel(walks, L, num_samples)
        
        atm_counts = atm_counts.get()

        # Free GPU memory
        del walks
        cp.get_default_memory_pool().free_all_blocks()

        return np.mean(atm_counts), np.std(atm_counts) / np.sqrt(len(atm_counts))
    
    def estimate_mu(self, num_samples=10000, max_trial=10, pivot_steps_mode='single'):
        """Estimate μ using atmosphere statistics
        max_trial: max trials before pivot_move() fails to generate a new SAW and returns the same walk 
        pivot_steps_mode: 'single' (1 pivot) or 'multiple' (L pivots)
        """
        lengths = np.arange(4, self.L_max+1, 2)
        atm_means = []
        atm_stds = []
        
        print("Running SAW simulations for μ estimation on GPU...")
        start = time()
        for L in tqdm(lengths):
            pivot_steps = 1 if pivot_steps_mode == 'single' else L
            mean_atm, std_atm = self.simulate_atmosphere(L, num_samples, max_trial, pivot_steps=pivot_steps)
            atm_means.append(mean_atm)
            atm_stds.append(std_atm)
            print(f"L={L}: ⟨a⟩ = {mean_atm:.3f} ± {std_atm:.3f}")
        end = time()
        
        atm_means = np.array(atm_means)
        atm_stds = np.array(atm_stds)
        def model(n, mu, gamma, c):
            return mu * (1 + (gamma-1)/n + c/n**2)
        
        popt, pcov = curve_fit(model, lengths, atm_means)
        mu, gamma = popt[0], popt[1]
        
        # Plot 1: <a> vs L
        plt.figure(figsize=(10,6))
        plt.scatter(lengths, atm_means, label='Simulation Data')
        x_fit = np.linspace(lengths[0], lengths[-1], 100)
        plt.plot(x_fit, model(x_fit, *popt), 'r-', 
                label=f'Fit: μ={mu:.6f}, γ={gamma:.6f}')
        plt.xlabel('L')
        plt.ylabel('<a>')
        plt.title(f'Atmosphere Statistics for 2D SAWs (GPU Parallel), N={num_samples}, Time={end-start:.2f}s, mu~{mu:.4f}')
        plt.legend()
        plt.grid(True)
        plt.savefig('saw_mu_estimation_gpu_parallel_L.png', dpi=300)
        plt.close()
        
        # Plot 2: <a> vs 1/L with benchmark mu
        benchmark_mu = 2.638158  # Theoretical mu for 2D SAW
        inv_lengths = 1 / lengths
        plt.figure(figsize=(10,6))
        plt.scatter(inv_lengths, atm_means, label='Simulation Data')
        x_fit_inv = np.linspace(0, inv_lengths[0], 100)
        plt.plot(x_fit_inv, model(1/x_fit_inv, *popt), 'r-', 
                label=f'Fit: μ={mu:.6f}, γ={gamma:.6f}')
        plt.axhline(y=benchmark_mu, color='g', linestyle='--', label=f'Benchmark μ={benchmark_mu:.6f}')
        plt.xlabel('1/L')
        plt.ylabel('<a>')
        plt.title(f'Atmosphere Statistics for 2D SAWs: Convergence to μ (GPU Parallel), N={num_samples}, Time={end-start:.2f}s, mu~{mu:.4f}')
        plt.legend()
        plt.grid(True)
        plt.savefig('saw_mu_estimation_gpu_parallel_inv_L.png', dpi=300)
        plt.close()
        
        return mu, gamma

# Benchmarking
if __name__ == "__main__":
    simulator = SAWSimulatorGPU(L_max=1000)
    
    # Test both modes
    for mode in ['single', 'multiple']:
        print(f"\nTesting mode: {mode}")
        start_time = time()
        mu, gamma = simulator.estimate_mu(num_samples=10000, max_trial=10, pivot_steps_mode=mode)
        gpu_time = time() - start_time
        
        print(f"GPU Parallel Version Results (Mode: {mode}):")
        print(f"Estimated μ = {mu:.6f}")
        print(f"Estimated γ = {gamma:.6f}")
        print(f"Computation Time = {gpu_time:.2f} seconds")

Grid size: 2001, Batch size: 256

Testing mode: single
Running SAW simulations for μ estimation on GPU...


  0%|          | 1/499 [00:12<1:44:47, 12.63s/it]

L=4: ⟨a⟩ = 2.750 ± 0.004


  0%|          | 2/499 [00:25<1:43:31, 12.50s/it]

L=6: ⟨a⟩ = 2.897 ± 0.003


...

L=30: ⟨a⟩ = 2.936 ± 0.003


  3%|▎         | 15/499 [04:23<2:58:49, 22.17s/it]

L=32: ⟨a⟩ = 2.957 ± 0.002


  3%|▎         | 16/499 [04:49<3:07:15, 23.26s/it]

L=34: ⟨a⟩ = 2.945 ± 0.003


### 2. Recursive method

### Key implementation details:
1. to prevent the value $C_{L1} \times C_{L2}$ from going too large, exceeding the range of float64, we use $log(C_{L1+L2}) = log(C_{L1}) + log(C_{L2}) + log(\hat{B})$ instead.
2. we skip the points where $\hat{B} = 0$ when doing curve_fit.

#### 1) Continuous L Sampling Approach

In [3]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from collections import defaultdict
from scipy.optimize import curve_fit

# Define the CUDA kernel for checking if concatenated walks are SAWs
check_saw_kernel = cp.RawKernel(r'''
extern "C" __global__
void check_saw_kernel(
    int* walks,  // Flattened array: [num_samples, L+1, 2]
    int num_samples, int L, int grid_size,
    int* counts,  // Global array: [num_samples, grid_size * grid_size]
    int* is_saw_flags  // Output: 1 if the walk is a SAW, 0 if collision detected
) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid >= num_samples) return;

    int walk_offset = tid * (L + 1) * 2;
    int count_offset = tid * grid_size * grid_size;

    // Build counts for the walk
    bool has_collision = false;
    for (int i = 0; i <= L; i++) {
        int x = walks[walk_offset + i * 2];
        int y = walks[walk_offset + i * 2 + 1];
        if (x < 0 || x >= grid_size || y < 0 || y >= grid_size) {
            has_collision = true;
            break;
        }
        int idx = count_offset + (x * grid_size + y);
        atomicAdd(&counts[idx], 1);
        if (counts[idx] > 1) {
            has_collision = true;
            break;
        }
    }

    // Reset counts
    for (int i = 0; i <= L; i++) {
        int x = walks[walk_offset + i * 2];
        int y = walks[walk_offset + i * 2 + 1];
        int idx = count_offset + (x * grid_size + y);
        counts[idx] = 0;
    }

    is_saw_flags[tid] = (has_collision ? 0 : 1);
}
''', 'check_saw_kernel')

# Reusing the pivot kernel from the previous implementation
pivot_kernel = cp.RawKernel(r'''
extern "C" __global__
void pivot_move_kernel(
    int* walks,  // Flattened array: [batch_size, L+1, 2]
    int batch_size, int L, int grid_size, int max_trial,
    int* counts,  // Global array: [batch_size, grid_size * grid_size]
    int* k_values,  // Random pivot points
    int* g_values,  // Random symmetry operations
    int* collision_flags  // Output: 1 if successful pivot, 0 if failed
) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid >= batch_size) return;

    int walk_offset = tid * (L + 1) * 2;
    int count_offset = tid * grid_size * grid_size;

    // Get random pivot point and symmetry operation
    int k = k_values[tid];
    int g = g_values[tid];

    // Try pivot move up to max_trial times
    for (int trial = 0; trial < max_trial; trial++) {
        // Copy walk to temporary array
        int temp_walk[2002][2];
        for (int i = 0; i <= L; i++) {
            temp_walk[i][0] = walks[walk_offset + i * 2];
            temp_walk[i][1] = walks[walk_offset + i * 2 + 1];
        }

        // Apply symmetry operation to subwalk (from k to L)
        int pivot_x = temp_walk[k][0];
        int pivot_y = temp_walk[k][1];
        for (int i = k; i <= L; i++) {
            int dx = temp_walk[i][0] - pivot_x;
            int dy = temp_walk[i][1] - pivot_y;
            if (g == 0) {  // rotate90
                temp_walk[i][0] = pivot_x - dy;
                temp_walk[i][1] = pivot_y + dx;
            } else if (g == 1) {  // rotate180
                temp_walk[i][0] = pivot_x - dx;
                temp_walk[i][1] = pivot_y - dy;
            } else if (g == 2) {  // reflect_x
                temp_walk[i][0] = pivot_x + dx;
                temp_walk[i][1] = pivot_y - dy;
            } else {  // reflect_y
                temp_walk[i][0] = pivot_x - dx;
                temp_walk[i][1] = pivot_y + dy;
            }
        }

        // Collision check 
        bool has_collision = false;
        for (int i = 0; i <= L; i++) {
            int x = temp_walk[i][0];
            int y = temp_walk[i][1];
            if (x < 0 || x >= grid_size || y < 0 || y >= grid_size) {
                has_collision = true;
                break;
            }
            int idx = count_offset + (x * grid_size + y);
            atomicAdd(&counts[idx], 1);
            if (counts[idx] > 1) {
                has_collision = true;
                break;
            }
        }

        // Reset counts and update walk if no collision
        if (!has_collision) {
            for (int i = 0; i <= L; i++) {
                int x = temp_walk[i][0];
                int y = temp_walk[i][1];
                int idx = count_offset + (x * grid_size + y);
                counts[idx] = 0;
            }
            for (int i = 0; i <= L; i++) {
                walks[walk_offset + i * 2] = temp_walk[i][0];
                walks[walk_offset + i * 2 + 1] = temp_walk[i][1];
            }
            collision_flags[tid] = 1;
            return;
        } else {
            for (int i = 0; i <= L; i++) {
                int x = temp_walk[i][0];
                int y = temp_walk[i][1];
                int idx = count_offset + (x * grid_size + y);
                counts[idx] = 0;
            }
            // Update k and g for next trial
            k = (k + 1) % (L - 1);
            if (k == 0) k = 1;
            g = (g + 1) % 4;
        }
    }
    collision_flags[tid] = 0;
}
''', 'pivot_move_kernel')

MU_ESTIMATE = 2.638

class SAWSimulatorGPU:
    def __init__(self, L_max=1000):
        self.directions = np.array([[1,0], [-1,0], [0,1], [0,-1]])
        self.L_max = L_max
        self.grid_size = 2 * L_max + 1
        self.batch_size = 256
        self.known_counts = {
            1: np.log(4), 2: np.log(12), 3: np.log(36), 4: np.log(100), 5: np.log(284),
            6: np.log(780), 7: np.log(2172), 8: np.log(5916), 9: np.log(16268),
            10: np.log(44100), 11: np.log(120292), 12: np.log(324932), 13: np.log(881500),
            14: np.log(2374444), 15: np.log(6416596), 16: np.log(17245332), 17: np.log(46466676),
            18: np.log(124658732), 19: np.log(335116620), 20: np.log(897697164),
            21: np.log(2408806028), 22: np.log(6444560484), 23: np.log(17266613812),
            24: np.log(46146397316), 25: np.log(123481354908), 26: np.log(329712786220),
            27: np.log(881317491628)
        }
        self.B_estimates = defaultdict(list)
        print(f"Grid size: {self.grid_size}, Batch size: {self.batch_size}")

    def generate_initial_saw(self, L):
        start_x, start_y = self.L_max, self.L_max
        walk = np.array([[start_x + i, start_y] for i in range(L+1)], dtype=np.int32)
        return walk

    def generate_multiple_saws(self, base_walk, num_samples):
        walks = np.tile(base_walk[np.newaxis, :, :], (num_samples, 1, 1))
        return cp.array(walks)

    def pivot_move_single(self, walk, L, max_trial=10):
        for _ in range(max_trial):
            k = np.random.randint(1, L)
            g = np.random.randint(0, 4)
            subwalk = walk[k:] - walk[k]
            if g == 0:  # rotate90
                transformed = np.array([[-y, x] for [x, y] in subwalk])
            elif g == 1:  # rotate180
                transformed = np.array([[-x, -y] for [x, y] in subwalk])
            elif g == 2:  # reflect_x
                transformed = np.array([[x, -y] for [x, y] in subwalk])
            else:  # reflect_y
                transformed = np.array([[-x, y] for [x, y] in subwalk])
            new_walk = np.concatenate([walk[:k], walk[k] + transformed])
            indices = new_walk[:, 0] * self.grid_size + new_walk[:, 1]
            counts = np.bincount(indices, minlength=self.grid_size * self.grid_size)
            if np.all(counts <= 1):
                return new_walk, True
        return walk, False

    def thermalize(self, walk, L):
        for _ in range(10 * L):
            walk, _ = self.pivot_move_single(walk, L)
        return walk

    def pivot_move_parallel(self, walks, L, max_trial):
        num_samples = walks.shape[0]
        batch_size = self.batch_size
        all_collision_flags = cp.zeros(num_samples, dtype=cp.int32)

        for batch_start in range(0, num_samples, batch_size):
            batch_end = min(batch_start + batch_size, num_samples)
            current_batch_size = batch_end - batch_start

            batch_walks = walks[batch_start:batch_end]
            batch_walks_flat = batch_walks.reshape(current_batch_size * (L + 1) * 2)
            collision_flags = cp.zeros(current_batch_size, dtype=cp.int32)

            counts = cp.zeros(current_batch_size * self.grid_size * self.grid_size, dtype=cp.int32)
            k_values = cp.random.randint(1, L, size=current_batch_size, dtype=cp.int32)
            g_values = cp.random.randint(0, 4, size=current_batch_size, dtype=cp.int32)

            threads_per_block = min(256, current_batch_size)
            blocks_per_grid = (current_batch_size + threads_per_block - 1) // threads_per_block

            pivot_kernel(
                (blocks_per_grid,), (threads_per_block,),
                (batch_walks_flat, current_batch_size, L, self.grid_size, max_trial, counts, k_values, g_values, collision_flags)
            )

            walks[batch_start:batch_end] = batch_walks_flat.reshape(current_batch_size, L + 1, 2)
            all_collision_flags[batch_start:batch_end] = collision_flags

            del batch_walks, batch_walks_flat, collision_flags, counts, k_values, g_values
            cp.get_default_memory_pool().free_all_blocks()

        return walks, all_collision_flags

    def check_saw_parallel(self, walks, L, num_samples):
        batch_size = self.batch_size
        all_is_saw_flags = cp.zeros(num_samples, dtype=cp.int32)

        for batch_start in range(0, num_samples, batch_size):
            batch_end = min(batch_start + batch_size, num_samples)
            current_batch_size = batch_end - batch_start

            batch_walks = walks[batch_start:batch_end]
            batch_walks_flat = batch_walks.reshape(current_batch_size * (L + 1) * 2)
            is_saw_flags = cp.zeros(current_batch_size, dtype=cp.int32)
            counts = cp.zeros(current_batch_size * self.grid_size * self.grid_size, dtype=cp.int32)

            threads_per_block = min(256, current_batch_size)
            blocks_per_grid = (current_batch_size + threads_per_block - 1) // threads_per_block

            check_saw_kernel(
                (blocks_per_grid,), (threads_per_block,),
                (batch_walks_flat, current_batch_size, L, self.grid_size, counts, is_saw_flags)
            )

            all_is_saw_flags[batch_start:batch_end] = is_saw_flags

            del batch_walks, batch_walks_flat, is_saw_flags, counts
            cp.get_default_memory_pool().free_all_blocks()

        return all_is_saw_flags

    def estimate_B(self, L1, L2, num_samples, max_trial=10):
        # Generate initial walks for L1 and L2
        walk1_base = self.generate_initial_saw(L1)
        walk2_base = self.generate_initial_saw(L2)

        # Thermalize the base walks on CPU
        walk1_base = self.thermalize(walk1_base, L1)
        walk2_base = self.thermalize(walk2_base, L2)

        # Generate num_samples copies of each walk on GPU
        walks1 = self.generate_multiple_saws(walk1_base, num_samples)
        walks2 = self.generate_multiple_saws(walk2_base, num_samples)

        # Perform pivot moves in parallel
        walks1, _ = self.pivot_move_parallel(walks1, L1, max_trial)
        walks2, _ = self.pivot_move_parallel(walks2, L2, max_trial)

        # Concatenate the walks: walk1 + (walk2[1:] shifted to connect with walk1)
        concatenated_walks = cp.zeros((num_samples, L1 + L2 + 1, 2), dtype=cp.int32)
        for i in range(num_samples):
            walk1 = walks1[i]
            walk2 = walks2[i]
            offset = walk1[-1] - walk2[0] + cp.array([1, 0], dtype=cp.int32)
            concatenated_walks[i, :L1+1] = walk1
            concatenated_walks[i, L1+1:] = walk2[1:] + offset

        # Check if concatenated walks are SAWs in parallel
        L_total = L1 + L2
        is_saw_flags = self.check_saw_parallel(concatenated_walks, L_total, num_samples)

        # Calculate B_hat as the proportion of valid SAWs, add a small epsilon to avoid log(0)
        B_hat = cp.mean(is_saw_flags).get()
        B_hat = max(B_hat, 1e-10)  # Avoid log(0) by setting a minimum value
        self.B_estimates[(L1, L2)].append(B_hat)

        # Free GPU memory
        del walks1, walks2, concatenated_walks, is_saw_flags
        cp.get_default_memory_pool().free_all_blocks()

        return B_hat

    def recursive_estimate(self, L, num_samples=10000):
        if L in self.known_counts:
            return self.known_counts[L]
            
        L1 = max(k for k in self.known_counts if k <= L//2)
        L2 = L - L1
        log_c_L2 = self.recursive_estimate(L2, num_samples) if L2 not in self.known_counts else self.known_counts[L2]
        B_est = self.estimate_B(L1, L2, num_samples)
        log_c_L1 = self.known_counts[L1]
        # Compute log(c_L) = log(B_est) + log(c_L1) + log(c_L2)
        log_c_L = np.log(B_est) + log_c_L1 + log_c_L2
        
        self.known_counts[L] = log_c_L
        return log_c_L

    def estimate_mu(self, num_samples=10000):
        # Compute counts for all L up to L_max
        start = time()
        for L in tqdm(range(1, self.L_max + 1), desc="Estimating counts"):
            if L not in self.known_counts:
                self.recursive_estimate(L, num_samples)
        end = time()
        
        # Prepare data for fitting
        L_list = np.array(sorted(self.known_counts.keys()))
        log_c_N = np.array([self.known_counts[L] for L in L_list])
        # Skip entries where log_c_N is inf or nan
        valid_indices = np.isfinite(log_c_N)
        L_list = L_list[valid_indices]
        log_c_N = log_c_N[valid_indices]
        if len(L_list) < 3:
            raise ValueError("Not enough valid data points for fitting after removing inf/NaN")
        
        # Fitting function
        def fitting_func(N, log_mu, A, gamma):
            return N * log_mu + (gamma - 1) * np.log(N) + A
        
        # Curve fitting
        p0 = [np.log(MU_ESTIMATE), 1.0, 1.344]
        bounds = ([0, -np.inf, 0], [np.inf, np.inf, np.inf])
        popt, _ = curve_fit(fitting_func, L_list, log_c_N, p0=p0, bounds=bounds)
        mu_est = np.exp(popt[0])
        
        # Plotting
        mu_N = [np.exp(log_c / L) for L, log_c in sorted(self.known_counts.items())]
        
        plt.figure(figsize=(10, 6))
        plt.plot(L_list, mu_N[:len(L_list)], 'bo-', label='μ_N')
        plt.axhline(y=MU_ESTIMATE, color="k", linestyle="--", label=f"Theoretical μ ≈ {MU_ESTIMATE}")
        plt.xlabel('L')
        plt.ylabel('μ_N')
        plt.title(f'Recursive Method Estimation(GPU), N={num_samples}, Time={end-start:.2f}s, mu~{mu_est:.4f}')
        plt.legend()
        plt.grid(True)
        plt.savefig('recursive_mu_vs_l.png')
        plt.close()
        
        return mu_est

# Main execution
if __name__ == "__main__":
    simulator = SAWSimulatorGPU(L_max=200)
    start_time = time()
    mu_final = simulator.estimate_mu(num_samples=10000)
    gpu_time = time() - start_time
    print(f"\nGPU Recursive Method Results:")
    print(f"Final μ estimate = {mu_final:.7f}")
    print(f"Computation Time = {gpu_time:.2f} seconds")

Grid size: 401, Batch size: 256


Estimating counts: 100%|██████████| 200/200 [08:03<00:00,  2.42s/it]


GPU Recursive Method Results:
Final μ estimate = 2.6200326
Computation Time = 483.58 seconds


#### 2) Discrete L Sampling Approach

### Key implementation details
1. discrete L sampling allows further speed-up as L1=L2, thus only one base thermolisation is needed before both walks start doing parallel pivot moves to this same base walk.

In [2]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
from collections import defaultdict
from scipy.optimize import curve_fit

# Define the CUDA kernel for checking if concatenated walks are SAWs
check_saw_kernel = cp.RawKernel(r'''
extern "C" __global__
void check_saw_kernel(
    int* walks,  // Flattened array: [num_samples, L+1, 2]
    int num_samples, int L, int grid_size,
    int* counts,  // Global array: [num_samples, grid_size * grid_size]
    int* is_saw_flags  // Output: 1 if the walk is a SAW, 0 if collision detected
) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid >= num_samples) return;

    int walk_offset = tid * (L + 1) * 2;
    int count_offset = tid * grid_size * grid_size;

    // Build counts for the walk
    bool has_collision = false;
    for (int i = 0; i <= L; i++) {
        int x = walks[walk_offset + i * 2];
        int y = walks[walk_offset + i * 2 + 1];
        if (x < 0 || x >= grid_size || y < 0 || y >= grid_size) {
            has_collision = true;
            break;
        }
        int idx = count_offset + (x * grid_size + y);
        atomicAdd(&counts[idx], 1);
        if (counts[idx] > 1) {
            has_collision = true;
            break;
        }
    }

    // Reset counts
    for (int i = 0; i <= L; i++) {
        int x = walks[walk_offset + i * 2];
        int y = walks[walk_offset + i * 2 + 1];
        if (x >= 0 && x < grid_size && y >= 0 && y < grid_size) {
            int idx = count_offset + (x * grid_size + y);
            counts[idx] = 0;
        }
    }

    is_saw_flags[tid] = (has_collision ? 0 : 1);
}
''', 'check_saw_kernel')

# Reusing the pivot kernel from the previous implementation
pivot_kernel = cp.RawKernel(r'''
extern "C" __global__
void pivot_move_kernel(
    int* walks,  // Flattened array: [batch_size, L+1, 2]
    int batch_size, int L, int grid_size, int max_trial,
    int* counts,  // Global array: [batch_size, grid_size * grid_size]
    int* k_values,  // Random pivot points
    int* g_values,  // Random symmetry operations
    int* collision_flags  // Output: 1 if successful pivot, 0 if failed
) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid >= batch_size) return;

    int walk_offset = tid * (L + 1) * 2;
    int count_offset = tid * grid_size * grid_size;

    // Get random pivot point and symmetry operation
    int k = k_values[tid];
    int g = g_values[tid];

    // Try pivot move up to max_trial times
    for (int trial = 0; trial < max_trial; trial++) {
        // Copy walk to temporary array
        int temp_walk[2002][2];
        for (int i = 0; i <= L; i++) {
            temp_walk[i][0] = walks[walk_offset + i * 2];
            temp_walk[i][1] = walks[walk_offset + i * 2 + 1];
        }

        // Apply symmetry operation to subwalk (from k to L)
        int pivot_x = temp_walk[k][0];
        int pivot_y = temp_walk[k][1];
        for (int i = k; i <= L; i++) {
            int dx = temp_walk[i][0] - pivot_x;
            int dy = temp_walk[i][1] - pivot_y;
            if (g == 0) {  // rotate90
                temp_walk[i][0] = pivot_x - dy;
                temp_walk[i][1] = pivot_y + dx;
            } else if (g == 1) {  // rotate180
                temp_walk[i][0] = pivot_x - dx;
                temp_walk[i][1] = pivot_y - dy;
            } else if (g == 2) {  // reflect_x
                temp_walk[i][0] = pivot_x + dx;
                temp_walk[i][1] = pivot_y - dy;
            } else {  // reflect_y
                temp_walk[i][0] = pivot_x - dx;
                temp_walk[i][1] = pivot_y + dy;
            }
        }

        // Collision check 
        bool has_collision = false;
        for (int i = 0; i <= L; i++) {
            int x = temp_walk[i][0];
            int y = temp_walk[i][1];
            if (x < 0 || x >= grid_size || y < 0 || y >= grid_size) {
                has_collision = true;
                break;
            }
            int idx = count_offset + (x * grid_size + y);
            atomicAdd(&counts[idx], 1);
            if (counts[idx] > 1) {
                has_collision = true;
                break;
            }
        }

        // Reset counts and update walk if no collision
        if (!has_collision) {
            for (int i = 0; i <= L; i++) {
                int x = temp_walk[i][0];
                int y = temp_walk[i][1];
                if (x >= 0 && x < grid_size && y >= 0 && y < grid_size) {
                    int idx = count_offset + (x * grid_size + y);
                    counts[idx] = 0;
                }
            }
            for (int i = 0; i <= L; i++) {
                walks[walk_offset + i * 2] = temp_walk[i][0];
                walks[walk_offset + i * 2 + 1] = temp_walk[i][1];
            }
            collision_flags[tid] = 1;
            return;
        } else {
            for (int i = 0; i <= L; i++) {
                int x = temp_walk[i][0];
                int y = temp_walk[i][1];
                if (x >= 0 && x < grid_size && y >= 0 && y < grid_size) {
                    int idx = count_offset + (x * grid_size + y);
                    counts[idx] = 0;
                }
            }
            // Update k and g for next trial
            k = (k + 1) % (L - 1);
            if (k == 0) k = 1;
            g = (g + 1) % 4;
        }
    }
    collision_flags[tid] = 0;
}
''', 'pivot_move_kernel')

MU_ESTIMATE = 2.638

class SAWSimulatorGPU:
    def __init__(self, L_max=1000):
        self.directions = np.array([[1,0], [-1,0], [0,1], [0,-1]])
        self.L_max = L_max
        self.max_grid_size = 2 * L_max + 1
        self.batch_size = 16  # Reduced to lower memory usage
        self.known_counts = {
            1: np.log(4), 2: np.log(12), 3: np.log(36), 4: np.log(100), 5: np.log(284),
            6: np.log(780), 7: np.log(2172), 8: np.log(5916), 9: np.log(16268),
            10: np.log(44100), 11: np.log(120292), 12: np.log(324932), 13: np.log(881500),
            14: np.log(2374444), 15: np.log(6416596), 16: np.log(17245332), 17: np.log(46466676),
            18: np.log(124658732), 19: np.log(335116620), 20: np.log(897697164),
            21: np.log(2408806028), 22: np.log(6444560484), 23: np.log(17266613812),
            24: np.log(46146397316), 25: np.log(123481354908), 26: np.log(329712786220),
            27: np.log(881317491628)
        }
        self.B_estimates = defaultdict(list)
        print(f"Max grid size: {self.max_grid_size}, Batch size: {self.batch_size}")

    def generate_initial_saw(self, L, grid_size):
        start_x, start_y = grid_size // 2, grid_size // 2
        walk = np.array([[start_x + i, start_y] for i in range(L+1)], dtype=np.int32)
        return walk

    def generate_multiple_saws(self, base_walk, num_samples):
        walks = np.tile(base_walk[np.newaxis, :, :], (num_samples, 1, 1))
        return cp.array(walks)

    def pivot_move_single(self, walk, L, max_trial=10):
        for _ in range(max_trial):
            k = np.random.randint(1, L)
            g = np.random.randint(0, 4)
            subwalk = walk[k:] - walk[k]
            if g == 0:  # rotate90
                transformed = np.array([[-y, x] for [x, y] in subwalk])
            elif g == 1:  # rotate180
                transformed = np.array([[-x, -y] for [x, y] in subwalk])
            elif g == 2:  # reflect_x
                transformed = np.array([[x, -y] for [x, y] in subwalk])
            else:  # reflect_y
                transformed = np.array([[-x, y] for [x, y] in subwalk])
            new_walk = np.concatenate([walk[:k], walk[k] + transformed])
            # Check for collisions using np.unique
            unique_points = np.unique(new_walk, axis=0)
            if len(unique_points) == len(new_walk):  # All points are unique, no collisions
                return new_walk, True
        return walk, False

    def thermalize(self, walk, L):
        for _ in range(10 * L):
            walk, _ = self.pivot_move_single(walk, L)
        return walk

    def pivot_move_parallel(self, walks, L, max_trial, grid_size):
        num_samples = walks.shape[0]
        batch_size = self.batch_size
        all_collision_flags = cp.zeros(num_samples, dtype=cp.int32)

        # Free memory before large allocation
        cp.get_default_memory_pool().free_all_blocks()

        for batch_start in range(0, num_samples, batch_size):
            batch_end = min(batch_start + batch_size, num_samples)
            current_batch_size = batch_end - batch_start

            batch_walks = walks[batch_start:batch_end]
            batch_walks_flat = batch_walks.reshape(current_batch_size * (L + 1) * 2)
            collision_flags = cp.zeros(current_batch_size, dtype=cp.int32)

            counts = cp.zeros(current_batch_size * grid_size * grid_size, dtype=cp.int32)
            k_values = cp.random.randint(1, L, size=current_batch_size, dtype=cp.int32)
            g_values = cp.random.randint(0, 4, size=current_batch_size, dtype=cp.int32)

            threads_per_block = min(256, current_batch_size)
            blocks_per_grid = (current_batch_size + threads_per_block - 1) // threads_per_block

            pivot_kernel(
                (blocks_per_grid,), (threads_per_block,),
                (batch_walks_flat, current_batch_size, L, grid_size, max_trial, counts, k_values, g_values, collision_flags)
            )

            walks[batch_start:batch_end] = batch_walks_flat.reshape(current_batch_size, L + 1, 2)
            all_collision_flags[batch_start:batch_end] = collision_flags

            del batch_walks, batch_walks_flat, collision_flags, counts, k_values, g_values
            cp.get_default_memory_pool().free_all_blocks()

        return walks, all_collision_flags

    def check_saw_parallel(self, walks, L, num_samples, grid_size):
        batch_size = self.batch_size
        all_is_saw_flags = cp.zeros(num_samples, dtype=cp.int32)

        # Free memory before large allocation
        cp.get_default_memory_pool().free_all_blocks()

        for batch_start in range(0, num_samples, batch_size):
            batch_end = min(batch_start + batch_size, num_samples)
            current_batch_size = batch_end - batch_start

            batch_walks = walks[batch_start:batch_end]
            batch_walks_flat = batch_walks.reshape(current_batch_size * (L + 1) * 2)
            is_saw_flags = cp.zeros(current_batch_size, dtype=cp.int32)
            counts = cp.zeros(current_batch_size * grid_size * grid_size, dtype=cp.int32)

            threads_per_block = min(256, current_batch_size)
            blocks_per_grid = (current_batch_size + threads_per_block - 1) // threads_per_block

            check_saw_kernel(
                (blocks_per_grid,), (threads_per_block,),
                (batch_walks_flat, current_batch_size, L, grid_size, counts, is_saw_flags)
            )

            all_is_saw_flags[batch_start:batch_end] = is_saw_flags

            del batch_walks, batch_walks_flat, is_saw_flags, counts
            cp.get_default_memory_pool().free_all_blocks()

        return all_is_saw_flags

    def estimate_B(self, L1, L2, num_samples, max_trial=10):
        # Since L1 == L2 == L in this context, use a single thermalized walk
        L = L1  # L1 == L2
        # Set grid size based on current L
        grid_size = 2 * L + 1
        # Generate and thermalize a single base walk
        base_walk = self.generate_initial_saw(L, grid_size)
        base_walk = self.thermalize(base_walk, L)

        # Generate two sets of num_samples walks from the same thermalized base walk
        walks1 = self.generate_multiple_saws(base_walk, num_samples)
        walks2 = self.generate_multiple_saws(base_walk, num_samples)

        # Perform pivot moves in parallel on both sets
        walks1, _ = self.pivot_move_parallel(walks1, L, max_trial, grid_size)
        walks2, _ = self.pivot_move_parallel(walks2, L, max_trial, grid_size)

        # Concatenate the walks: walk1 + (walk2[1:] shifted to connect with walk1)
        cp.get_default_memory_pool().free_all_blocks()  # Free memory before large allocation
        concatenated_walks = cp.zeros((num_samples, L1 + L2 + 1, 2), dtype=cp.int32)
        for i in range(num_samples):
            walk1 = walks1[i]
            walk2 = walks2[i]
            offset = walk1[-1] - walk2[0] + cp.array([1, 0], dtype=cp.int32)
            concatenated_walks[i, :L1+1] = walk1
            concatenated_walks[i, L1+1:] = walk2[1:] + offset

        # Check if concatenated walks are SAWs in parallel
        L_total = L1 + L2
        concat_grid_size = 2 * L_total + 1
        is_saw_flags = self.check_saw_parallel(concatenated_walks, L_total, num_samples, concat_grid_size)

        # Calculate B_hat as the proportion of valid SAWs, add a small epsilon to avoid log(0)
        B_hat = cp.mean(is_saw_flags).get()
        B_hat = max(B_hat, 1e-10)  # Avoid log(0) by setting a minimum value
        self.B_estimates[(L1, L2)].append(B_hat)

        # Free GPU memory
        del walks1, walks2, concatenated_walks, is_saw_flags
        cp.get_default_memory_pool().free_all_blocks()

        return B_hat

    def recursive_estimate(self, L, num_samples=10000):
        if L in self.known_counts:
            return self.known_counts[L]
            
        L1 = max(k for k in self.known_counts if k <= L//2)
        L2 = L - L1
        log_c_L2 = self.recursive_estimate(L2, num_samples) if L2 not in self.known_counts else self.known_counts[L2]
        B_est = self.estimate_B(L1, L2, num_samples)
        log_c_L1 = self.known_counts[L1]
        # Compute log(c_L) = log(B_est) + log(c_L1) + log(c_L2)
        log_c_L = np.log(B_est) + log_c_L1 + log_c_L2
        
        self.known_counts[L] = log_c_L
        return log_c_L

    def estimate_mu(self, num_samples=10000):
        # Generate geometric sequence for L: 10, 20, 40, ..., up to L_max
        L_list = []
        L = 10
        while L <= self.L_max:
            L_list.append(L)
            L *= 2
        L_list = np.array(L_list)
        
        # Compute counts for L in the geometric sequence
        start = time()
        for L in tqdm(L_list, desc="Estimating counts"):
            if L not in self.known_counts:
                self.recursive_estimate(L, num_samples)
        end = time()
        
        # Prepare data for fitting
        log_c_N = np.array([self.known_counts[L] for L in L_list])
        # Skip entries where log_c_N is inf or nan
        valid_indices = np.isfinite(log_c_N)
        L_list = L_list[valid_indices]
        log_c_N = log_c_N[valid_indices]
        if len(L_list) < 3:
            raise ValueError("Not enough valid data points for fitting after removing inf/NaN")
        
        # Fitting function
        def fitting_func(N, log_mu, A, gamma):
            return N * log_mu + (gamma - 1) * np.log(N) + A
        
        # Curve fitting
        p0 = [np.log(MU_ESTIMATE), 1.0, 1.344]
        bounds = ([0, -np.inf, 0], [np.inf, np.inf, np.inf])
        popt, _ = curve_fit(fitting_func, L_list, log_c_N, p0=p0, bounds=bounds)
        mu_est = np.exp(popt[0])
        
        # Plotting
        mu_N = [np.exp(log_c / L) for L, log_c in sorted(self.known_counts.items()) if L in L_list]
        
        plt.figure(figsize=(10, 6))
        plt.plot(L_list, mu_N, 'bo-', label='μ_N')
        plt.axhline(y=MU_ESTIMATE, color="k", linestyle="--", label=f"Theoretical μ ≈ {MU_ESTIMATE}")
        plt.xlabel('L')
        plt.ylabel('μ_N')
        plt.title(f'Geometric Recursive Method (GPU, Optimized, Unique, Dynamic, LowMem), N={num_samples}, Time={end-start:.2f}s, mu~{mu_est:.4f}')
        plt.legend()
        plt.grid(True)
        plt.savefig('geometric_recursive_optimized_unique_dynamic_lowmem_mu_vs_l.png')
        plt.close()
        
        return mu_est

# Main execution
if __name__ == "__main__":
    simulator = SAWSimulatorGPU(L_max=2560)
    start_time = time()
    mu_final = simulator.estimate_mu(num_samples=10000)
    gpu_time = time() - start_time
    print(f"\nGPU Geometric Recursive Method (Optimized, Unique, Dynamic, LowMem) Results:")
    print(f"Final μ estimate = {mu_final:.7f}")
    print(f"Computation Time = {gpu_time:.2f} seconds")

Max grid size: 5121, Batch size: 16


Estimating counts: 100%|██████████| 9/9 [04:55<00:00, 32.85s/it]


GPU Geometric Recursive Method (Optimized, Unique, Dynamic, LowMem) Results:
Final μ estimate = 2.5926828
Computation Time = 295.82 seconds
